In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv('../input/train.csv')
train.head()

In [ ]:
test = pd.read_csv('../input/test.csv')
test.head()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
test.info()

In [ ]:
test.describe()

In [ ]:
X_train = (train.iloc[:,1:].values).astype('float32') 
y_train = train.iloc[:,0].values.astype('int32') 
X_test = test.values.astype('float32')

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
sns.countplot(y_train)

In [ ]:
plt.hist(y_train)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28)

In [ ]:
plt.imshow(X_train[7])

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28,1)
X_train.shape

In [ ]:
X_test = X_test.reshape(X_test.shape[0],28,28,1)
X_test.shape

In [ ]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)
def standardize(x): 
    return (x-mean_px)/std_px

In [ ]:
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers.core import Lambda
from keras.layers.core import Dense
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.utils.np_utils import to_categorical
from keras.models import  Sequential
from keras.preprocessing import image
from keras.optimizers import Adam

In [ ]:
y_train = to_categorical(y_train)

In [ ]:
model = Sequential([
    Lambda(standardize, input_shape=(28,28,1)),
    Conv2D(32,(3,3), activation='relu'),
    BatchNormalization(axis=1),
    Dropout(.5),
    Conv2D(32,(3,3), activation='relu'),
    BatchNormalization(axis=1),
    MaxPool2D(),
    Conv2D(64,(3,3), activation='relu'),
    BatchNormalization(axis=1),
    Dropout(.5),
    Conv2D(64,(3,3), activation='relu'),
    MaxPool2D(),
    Flatten(),
    BatchNormalization(),
    Dropout(.5),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(.5),
    Dense(10, activation='softmax')
    ])

In [ ]:
gen = image.ImageDataGenerator(rotation_range=10, 
                               width_shift_range=0.10, 
                               shear_range=0.5,
                               height_shift_range=0.10, 
                               zoom_range=0.10
                              )

In [ ]:
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.optimizer.lr=0.001
batches = gen.flow(X_train, y_train, batch_size=64)

In [ ]:
result = model.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=1)